In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import shapely.speedups
from tqdm import tqdm
from quetzal.model import stepmodel
from quetzal.engine import engine
from quetzal.io import excel

In [3]:
input_path = '../input/'
model_path = '../model/'

In [4]:
def average_speed(scenario):
    """Calculate the average speed on all connections of the road network."""
    # Load the zippedpickle file
    car = stepmodel.read_zippedpickles(model_path + scenario + '/' + 'de_road_los')
    
    # Load the car los to a DataFrame
    df = pd.DataFrame(car.car_los)
    
    # filter the DataFrame for origin, destination, length and in_vehicle_time
    df_all = pd.concat([df.origin, df.destination, df.length, df.in_vehicle_time], axis=1)
    
    # Calculate average speed from length and in_vehicle_time
    df_all['average_speed'] = (df_all['length']/1000) / (df_all['in_vehicle_time']/3600)
    
    # Sort the DataFrame
    df_all.sort_values(by=['origin', 'destination'], inplace=True, ignore_index=True)
    
    return df_all

In [5]:
# Define scenario here
scenario = "lim"
reference = "ref_35"
df_scen = average_speed(scenario)
df_ref = average_speed(reference)
print(df_ref.head(10))
print(df_scen.head(10))

car_los: 100%|████████████████████████████████████| 1/1 [00:02<00:00,  2.48s/it]


    origin destination  length  in_vehicle_time  average_speed
0  DE111_0     DE112_0   22344             1732      46.442494
1  DE111_0     DE112_1   40853             2543      57.833582
2  DE111_0     DE112_2   34180             2158      57.019462
3  DE111_0     DE112_3   30975             1977      56.403642
4  DE111_0     DE112_4   23409             2160      39.015000
5  DE111_0     DE112_5   26757             1526      63.122674
6  DE111_0     DE112_6   32618             1802      65.163596
7  DE111_0     DE113_0   23000             1572      52.671756
8  DE111_0     DE113_1   20393             1694      43.338135
9  DE111_0     DE113_2   40260             2494      58.113873
    origin destination  length  in_vehicle_time  average_speed
0  DE111_0     DE112_0   22344             2323      34.626948
1  DE111_0     DE112_1   40853             3011      48.844503
2  DE111_0     DE112_2   34180             2430      50.637037
3  DE111_0     DE112_3   30975             2435      45

In [6]:
# Rename columns
df_ref.rename(columns={"in_vehicle_time": "time_ref", "average_speed": "speed_ref"}, inplace=True)
df_scen.rename(columns={"in_vehicle_time": f"time_scen", "average_speed": f"speed_scen"}, inplace=True)

# Combine data from reference and scenario
df_all = pd.concat([df_ref, df_scen.speed_scen, df_scen.time_scen], axis=1)
print(df_all.head(20))

# Calculate difference in speed
df_all["speed_diff"] = df_all.speed_ref - df_all.speed_scen
diff = df_all[df_all.speed_diff > 0]

# Print results
print(f"Number of all connections: {len(df_all.index)}")
print(f"Different average speed in {len(diff.index)} connections")
print(df_all.head(20))

     origin destination  length  time_ref  speed_ref  speed_scen  time_scen
0   DE111_0     DE112_0   22344      1732  46.442494   34.626948       2323
1   DE111_0     DE112_1   40853      2543  57.833582   48.844503       3011
2   DE111_0     DE112_2   34180      2158  57.019462   50.637037       2430
3   DE111_0     DE112_3   30975      1977  56.403642   45.794661       2435
4   DE111_0     DE112_4   23409      2160  39.015000   29.947548       2814
5   DE111_0     DE112_5   26757      1526  63.122674   54.115281       1780
6   DE111_0     DE112_6   32618      1802  65.163596   56.781818       2068
7   DE111_0     DE113_0   23000      1572  52.671756   46.412556       1784
8   DE111_0     DE113_1   20393      1694  43.338135   31.437484       2289
9   DE111_0     DE113_2   40260      2494  58.113873   50.694648       2859
10  DE111_0     DE113_3   22141      1705  46.749326   39.517898       2017
11  DE111_0     DE113_4   38286      2821  48.858419   43.561820       3164
12  DE111_0 